In [1]:
from utils.tools import read_yaml
# GENERAL LIBRARIES 
import math
import numpy as np
import joblib
from pathlib import Path
# MACHINE LEARNING LIBRARIES
import sklearn
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
# OPTUNA
import optuna
from optuna.trial import TrialState
from utils.transformer import TransformerEncoder, PatchClassEmbedding, Patches
from utils.data import load_mpose, random_flip, random_noise, one_hot
from utils.tools import CustomSchedule, CosineSchedule
from utils.tools import Logger

2022-08-02 10:05:23.221345: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-02 10:05:23.221385: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/ubuntu/anaconda3/envs/act/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/anaconda3/envs/act/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.4.0 and strictly below 2.7.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.0 and 

In [2]:
config = read_yaml('utils/config.yaml')

print(config)
model_size = config['MODEL_SIZE']
n_heads = config[model_size]['N_HEADS']
n_layers = config[model_size]['N_LAYERS']
embed_dim = config[model_size]['EMBED_DIM']
dropout = config[model_size]['DROPOUT']
mlp_head_size = config[model_size]['MLP']
activation = tf.nn.gelu
d_model = 64 * n_heads
d_ff = d_model * 4

split = 1
fold = 0
trial = None
bin_path = config['MODEL_DIR']

{'GPU': 0, 'VERBOSE': True, 'MODEL_DIR': 'bin/', 'RESULTS_DIR': 'results/', 'LOG_DIR': 'logs/', 'MODEL_NAME': 'AcT', 'SPLITS': 3, 'FOLDS': 10, 'SEEDS': [11331], 'VAL_SIZE': 0.1, 'FRAMES': 30, 'CHANNELS': 4, 'DATASET': 'openpose', 'openpose': {'KEYPOINTS': 13}, 'posenet': {'KEYPOINTS': 17}, 'CLASSES': 20, 'LABELS': ['standing', 'check-watch', 'cross-arms', 'scratch-head', 'sit-down', 'get-up', 'turn-around', 'walking', 'wave1', 'boxing', 'kicking', 'pointing', 'pick-up', 'bending', 'hands-clapping', 'wave2', 'jogging', 'jumping', 'pjump', 'running'], 'FLIP_P': 0.5, 'RN_STD': 0.03, 'PATCH_SIZE': 1, 'MODEL_SIZE': 'micro', 'micro': {'N_HEADS': 1, 'N_LAYERS': 4, 'EMBED_DIM': 64, 'DROPOUT': 0.3, 'MLP': 256}, 'small': {'N_HEADS': 2, 'N_LAYERS': 5, 'EMBED_DIM': 128, 'DROPOUT': 0.3, 'MLP': 256}, 'base': {'N_HEADS': 3, 'N_LAYERS': 6, 'EMBED_DIM': 192, 'DROPOUT': 0.3, 'MLP': 256}, 'large': {'N_HEADS': 4, 'N_LAYERS': 6, 'EMBED_DIM': 256, 'DROPOUT': 0.4, 'MLP': 512}, 'N_EPOCHS': 1, 'BATCH_SIZE': 51

In [4]:
def get_data():
    X_train, y_train, X_test, y_test = load_mpose(config['DATASET'], split, verbose=False)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                        test_size=config['VAL_SIZE'],
                                                        random_state=42,
                                                        stratify=y_train)
            
    ds_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    ds_train = ds_train.map(lambda x,y : one_hot(x,y,config['CLASSES']), 
                            num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds_train = ds_train.cache()
    ds_train = ds_train.map(random_flip, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds_train = ds_train.map(random_noise, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds_train = ds_train.shuffle(X_train.shape[0])
    ds_train = ds_train.batch(config['BATCH_SIZE'])
    ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

    ds_val = tf.data.Dataset.from_tensor_slices((X_val, y_val))
    ds_val = ds_val.map(lambda x,y : one_hot(x,y,config['CLASSES']), 
                            num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds_val = ds_val.cache()
    ds_val = ds_val.batch(config['BATCH_SIZE'])
    ds_val = ds_val.prefetch(tf.data.experimental.AUTOTUNE)

    ds_test = tf.data.Dataset.from_tensor_slices((X_test, y_test))
    ds_test = ds_test.map(lambda x,y : one_hot(x,y,config['CLASSES']), 
                            num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds_test = ds_test.cache()
    ds_test = ds_test.batch(config['BATCH_SIZE'])
    ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)
    return ds_train, ds_test,ds_val,X_train

In [5]:
ds_train, ds_test,ds_val,x_train = get_data()

2022-08-02 10:05:59.204304: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-02 10:05:59.204337: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-02 10:05:59.204360: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ubuntu-ThinkPad-X250): /proc/driver/nvidia/version does not exist
2022-08-02 10:05:59.206839: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-02 10:05:59.212089: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 141086

In [6]:
X, y = tuple(zip(*ds_test))

In [23]:
x_demo=X[0][0]

In [8]:
x_train[0].shape

(30, 52)

In [ ]:
y_pred = np.argmax(tf.nn.softmax(model.predict(tf.concat(X, axis=0)), axis=-1),axis=1)

In [9]:
tf.expand_dims(x_train[0], axis=0)

<tf.Tensor: shape=(1, 30, 52), dtype=float64, numpy=
array([[[ 0.00448911, -0.36499393,  0.00448911, ...,  2.36464603,
          0.09099247,  2.36464603],
        [ 0.02615695, -0.36490139,  0.02166783, ...,  2.39078909,
          0.02501499,  0.02614306],
        [ 0.03384959, -0.37170456,  0.00769264, ...,  2.41642411,
          0.01352134,  0.02563502],
        ...,
        [ 0.03451963, -0.36856423,  0.01087998, ...,  2.29394845,
          0.01014142, -0.02388365],
        [ 0.02348415, -0.40197094, -0.01103548, ...,  2.32914274,
         -0.00631095,  0.03519429],
        [ 0.03274832, -0.40224001,  0.00926417, ...,  2.32244785,
          0.00881321, -0.00669489]]])>

In [10]:
def build_act(transformer):
        inputs = tf.keras.layers.Input(shape=(config['FRAMES'], 
                                              config[config['DATASET']]['KEYPOINTS']*config['CHANNELS']))
        x = tf.keras.layers.Dense(d_model)(inputs)
        x = PatchClassEmbedding(d_model, config['FRAMES'])(x)
        x = transformer(x)
        x = tf.keras.layers.Lambda(lambda x: x[:,0,:])(x)
        x = tf.keras.layers.Dense(mlp_head_size)(x)
        outputs = tf.keras.layers.Dense(config['CLASSES'])(x)
        return tf.keras.models.Model(inputs, outputs)


transformer = TransformerEncoder(d_model, n_heads,d_ff, dropout, activation, n_layers)
model = build_act(transformer)

print(model.summary())

#lr = CustomSchedule(d_model, 
#             warmup_steps=len(ds_train)*config['N_EPOCHS']*config['WARMUP_PERC'],
#             decay_step=len(ds_train)*config['N_EPOCHS']*config['STEP_PERC'])

optimizer = tfa.optimizers.AdamW(weight_decay=config['WEIGHT_DECAY'])

model.compile(optimizer=optimizer,
                           loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
                           metrics=[tf.keras.metrics.CategoricalAccuracy(name="accuracy")])


model.load_weights("bin/AcT_micro_3_9.h5")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30, 52)]          0         
                                                                 
 dense_24 (Dense)            (None, 30, 64)            3392      
                                                                 
 patch_class_embedding (Patc  (None, 31, 64)           2048      
 hClassEmbedding)                                                
                                                                 
 transformer_encoder (Transf  (None, 31, 64)           199936    
 ormerEncoder)                                                   
                                                                 
 lambda (Lambda)             (None, 64)                0         
                                                                 
 dense_25 (Dense)            (None, 256)               16640 

In [ ]:
y_pred = np.argmax(tf.nn.softmax(model.predict(tf.concat(X, axis=0)), axis=-1),axis=1)

In [11]:
y_demo=model.predict(tf.expand_dims(x_train[0], axis=0))

1/1 [==============================] - 2s 2s/step


In [12]:
y_demo.shape

(1, 20)

In [13]:
np.argmax(y_demo)

8

In [14]:
np.argmax(tf.nn.softmax(y_demo,axis=-1))

8

In [21]:
from mpose import MPOSE

In [22]:
dataset = MPOSE(pose_extractor="openpose", 
                split=split, 
                preprocess=None, 
                velocities=True, 
                remove_zip=False)

Initializing MPOSE2021 with openpose Pose Extractor
File exists in /home/ubuntu/.mpose/openpose.zip. specify overwrite=True if intended
Extracting Data...
File exists in /home/ubuntu/.mpose/openpose/. specify overwrite=True if intended


In [33]:
s=dataset.add_velocities(overwrite=True)

In [34]:
type(s)

NoneType

In [35]:
dataset.get_info()

----Dataset Information----
Pose Extractor: openpose
Split: 1
X_train shape: (12562, 30, 25, 5)
X_test shape: (2867, 30, 25, 5)
Min-Max feature ranges:
x: [0.0, 828.042]
y: [0.0, 558.729]
Vx: [-803.5, 790.405]
Vy: [-511.387, 555.379]
p: [0.0, 1.0]


In [36]:
dataset.remove_velocities()

In [48]:
X_train = np.load("/home/ubuntu/.mpose/openpose/1" + '/X_train.npy')

In [42]:
X_train[0][0].shape

(25, 3)

In [44]:
X_train.shape

(12562, 30, 25, 3)

In [50]:
def add_velocities(X_train,overwrite=False):
 
        
        seq_list = []
        for seq in X_train:
          
            v1 = np.zeros((30+1, seq.shape[1], 3-1))
            v2 = np.zeros((30+1, seq.shape[1], 3-1))
            
            v1[1:,...] = seq[:,:,:2]
            v2[:30,...] = seq[:,:,:2]
            vel = (v2-v1)[:-1,...]
            data = np.concatenate((seq[:,:,:2], vel), axis=-1)
            data = np.concatenate((data, seq[:,:,-1:]), axis=-1)       
            seq_list.append(data)
        X_train = np.stack(seq_list)

        return X_train

In [51]:
x_train=add_velocities(X_train)

In [52]:
x_train.shape

(12562, 30, 25, 5)

In [53]:
X_train.shape

(12562, 30, 25, 3)

In [59]:
x_train[0][0]

array([[0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00],
       [2.34870e+02, 6.64126e+01, 2.34870e+02, 6.64126e+01, 8.40771e-01],
       [2.49527e+02, 6.71612e+01, 2.49527e+02, 6.71612e+01, 8.75739e-01],
       [2.54645e+02, 1.00110e+02, 2.54645e+02, 1.00110e+02, 8.40024e-01],
       [2.60504e+02, 1.25009e+02, 2.60504e+02, 1.25009e+02, 8.68385e-01],
       [2.19502e+02, 6.49461e+01, 2.19502e+02, 6.49461e+01, 8.49073e-01],
       [2.18773e+02, 9.57001e+01, 2.18773e+02, 9.57001e+01, 7.49432e-01],
       [2.26101e+02, 1.09644e+02, 2.26101e+02, 1.09644e+02, 2.06299e-01],
       [2.34149e+02, 1.19892e+02, 2.34149e+02, 1.19892e+02, 6.39379e-01],
       [2.43674e+02, 1.20619e+02, 2.43674e+02, 1.20619e+02, 6.84198e-01],
       [2.45144e+02, 1.60173e+02, 2.45144e+02, 1.60173e+02, 7.38917e-01],
       [2.45136e+02, 2.00453e+02, 2.45136e+02, 2.00453e+02, 7.67821e-01],
       [2.25329e+02, 1.19862e+02, 2.25329e+02, 1.19862e+02, 6.53879e-01],
       [2.25361e+02, 1.55790e+02, 2.25